In [29]:
"""
    方差分析
"""
import numpy as np
import pandas as pd

In [30]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd
import numpy as np

d1 = pd.read_csv("data/材料1.csv")
d2 = pd.read_csv("data/材料2.csv")
d3 = pd.read_csv("data/材料3.csv")
d4 = pd.read_csv("data/材料4.csv")

d1['磁芯材料'] = '材料1'
d2['磁芯材料'] = '材料2'
d3['磁芯材料'] = '材料3'
d4['磁芯材料'] = '材料4'

data = pd.concat([d1, d2, d3, d4])

# data = data.drop(data.columns[-1], axis=1)  # 拼接 data 为一整个数据集


magnetic_flux_density = data.iloc[:, 4:-1].values

std_flux = np.std(magnetic_flux_density, axis=1)  # 标准差
max_flux = np.max(magnetic_flux_density, axis=1)  # 峰值

data['标准差'] = std_flux
data['峰值'] = max_flux

pd.set_option('future.no_silent_downcasting', True)

data['励磁波形'] = data['励磁波形'].replace({'正弦波': 1, '三角波': 2, '梯形波': 3})



# print(data.shape)
# print(data.head())

lst = ['温度', '频率', "磁芯损耗", '励磁波形']
for i in range(1024):
    lst.append(i)
lst.append('磁芯材料')
lst.append('标准差')
lst.append('峰值')
data.columns = lst


# to = pd.DataFrame(data)
# to.to_csv("~/Desktop/data.csv", index=False)

# filter_data = data[['温度', '频率', '励磁波形', '材料', '峰值', '磁芯损耗']]

data_updated = data[['温度', '励磁波形', '磁芯材料', '磁芯损耗']]


# 重命名列名以便于使用
data_updated.columns = ['温度', '励磁波形', '磁芯材料', '磁芯损耗']


# 初始化OneHotEncoder，使用'drop="first"'避免多重共线性
encoder = OneHotEncoder(drop="first")
material_encoded = encoder.fit_transform(data_updated[['磁芯材料']]).toarray()

# 将编码后的结果转换为DataFrame并重置索引
material_encoded_df = pd.DataFrame(material_encoded, columns=encoder.get_feature_names_out(['磁芯材料'])).reset_index(drop=True)

# 原数据集重置索引以确保匹配
data_updated = data_updated.reset_index(drop=True)

# 将编码后的列与原数据框（去掉原来的'材料'列）合并
data_encoded = pd.concat([data_updated.drop('磁芯材料', axis=1), material_encoded_df], axis=1)

In [33]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 将数据中的材料列组合成一个整体因素
data_encoded['磁芯材料'] = data_encoded[['磁芯材料_材料2', '磁芯材料_材料3', '磁芯材料_材料4']].idxmax(axis=1)

# 定义ANOVA模型
model = ols('磁芯损耗 ~ 温度 * 励磁波形 * 磁芯材料', data=data_encoded).fit()

# 执行ANOVA分析
anova_results = sm.stats.anova_lm(model, typ=2)

# 提取 ANOVA 结果并进行格式化
formatted_results = anova_results.to_string(header=True, float_format="{:.5e}".format)

# 打印格式化的 ANOVA 结果
print("ANOVA Results:")
print(formatted_results)

ANOVA Results:
                  sum_sq          df           F       PR(>F)
励磁波形         7.68135e+13 2.00000e+00 2.92412e+02 8.21893e-125
磁芯材料         3.57625e+13 2.00000e+00 1.36140e+02  3.27947e-59
励磁波形:磁芯材料    7.47067e+12 4.00000e+00 1.42196e+01  1.39409e-11
温度           8.22184e+12 1.00000e+00 6.25975e+01  2.75068e-15
温度:励磁波形      1.19876e+12 2.00000e+00 4.56342e+00  1.04439e-02
温度:磁芯材料      7.21134e+11 2.00000e+00 2.74520e+00  6.42747e-02
温度:励磁波形:磁芯材料 3.24387e+11 4.00000e+00 6.17435e-01  6.50071e-01
Residual     1.62631e+15 1.23820e+04         NaN          NaN


In [31]:
"""
    斯贝尔曼相关系数
"""
from scipy.stats import spearmanr

# 替换为您的实际数据
a_tempeture = filter_data.iloc[:, 0]
a_wire = filter_data.iloc[:, 1]
a_material = filter_data.iloc[:, 2]
loss = filter_data.iloc[:, -1]

# 计算斯皮尔曼相关系数
v1, p1 = spearmanr(a_tempeture, loss)
print(f"温度 相关系数: {v1}")

v2, p2 = spearmanr(a_wire, loss)
print(f"波形 相关系数: {v2}")

v3, p3 = spearmanr(a_material, loss)
print(f"材料 相关系数： {v3}")

v4, p4 = spearmanr(a_tempeture * a_wire, loss)
print(f"温度: 波形 相关系数: {v4}")

v5, p5 = spearmanr(a_tempeture * a_material, loss)
print(f"温度: 材料 相关系数: {v5}")

v6, p6 = spearmanr(a_material * a_wire, loss)
print(f"波形: 材料 相关系数: {v6}")

v7, p7 = spearmanr(a_material * a_wire * a_tempeture, loss)
print(f"温度: 波形: 材料 相关系数: {v7}")